# 1.2 EntropyMeasure - Randomized Measurement
## Multiple Experiments

Consider a scenario, you have multiple circuits that you want to run at once.

Call `.measure()` one by one will be inefficient,
no to mention that you also need to call `.anlyze()` for their post-processing.

Here we provide a more efficient way solve this problem,
where the true power of Qurrium as experiment manage toolkit.


### a. Import the instances


In [1]:
from qurry import EntropyMeasure

experiment_randomized = EntropyMeasure()
# It's default method. EntropyMeasure(method='randomized') also works

### b. Preparing quantum circuit

Prepare and add circuits to the `.wave` for later usage.


In [2]:
from qiskit import QuantumCircuit
from qurry.recipe import TrivialParamagnet, GHZ


def make_neel_circuit(n):
    qc = QuantumCircuit(n)
    for i in range(0, n, 2):
        qc.x(i)
    return qc


for i in range(2, 13, 2):
    experiment_randomized.add(TrivialParamagnet(i), f"trivial_paramagnet_{i}")
    experiment_randomized.add(GHZ(i), f"ghz_{i}")
    experiment_randomized.add(make_neel_circuit(i), f"neel_{i}")

experiment_randomized.waves

WaveContainer({
  'trivial_paramagnet_2': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x7800551fa030>,
  'ghz_2': <qurry.recipe.simple.cat.GHZ object at 0x780099863530>,
  'neel_2': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x78009a301c40>,
  'trivial_paramagnet_4': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x78009a2dd850>,
  'ghz_4': <qurry.recipe.simple.cat.GHZ object at 0x78005571eb70>,
  'neel_4': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x78005578ce30>,
  'trivial_paramagnet_6': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x7800556e1b80>,
  'ghz_6': <qurry.recipe.simple.cat.GHZ object at 0x7800551791f0>,
  'neel_6': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x78005530f4a0>,
  'trivial_paramagnet_8': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x78008418ce30>,
  'ghz_8': <qurry.recipe.simple.cat.GHZ object at 0x780055022ae0>,
  'neel_8': <qiskit.circuit.quantumcircuit.Quantu

### c. Execute multiple experiments at once

Let's demonstrate the true power of Qurrium.


In [3]:
from qurry.qurrent.randomized_measure.arguments import (
    EntropyMeasureRandomizedMeasureArgs,
)

Preparing a configuration list for multiple experiments with following parameters:

```python
class EntropyMeasureRandomizedMeasureArgs(total=False):
    """Output arguments for :meth:`output`."""
    shots: int
    """Number of shots."""
    tags: Optional[tuple[str, ...]]
    """The tags to be used for the experiment."""

    wave: Optional[Union[QuantumCircuit, Hashable]]
    """The key or the circuit to execute."""
    times: int
    """The number of random unitary operator. 
    It will denote as `N_U` in the experiment name."""
    measure: Optional[Union[tuple[int, int], int, list[int]]]
    """The measure range."""
    unitary_loc: Optional[Union[tuple[int, int], int, list[int]]]
    """The range of the unitary operator."""
    unitary_loc_not_cover_measure: bool
    """Whether the range of the unitary operator is not cover the measure range."""
    random_unitary_seeds: Optional[dict[int, dict[int, int]]]
    """The seeds for all random unitary operator.
    This argument only takes input as type of `dict[int, dict[int, int]]`.
    The first key is the index for the random unitary operator.
    The second key is the index for the qubit.

    .. code-block:: python
        {
            0: {0: 1234, 1: 5678},
            1: {0: 2345, 1: 6789},
            2: {0: 3456, 1: 7890},
        }

    If you want to generate the seeds for all random unitary operator,
    you can use the function :func:`generate_random_unitary_seeds` 
    in :mod:`qurry.qurrium.utils.random_unitary`.

    .. code-block:: python
        from qurry.qurrium.utils.random_unitary import generate_random_unitary_seeds

        random_unitary_seeds = generate_random_unitary_seeds(100, 2)
    """
```


In [4]:
config_list: list[EntropyMeasureRandomizedMeasureArgs] = [
    {
        "shots": 1024,
        "wave": f"{wave_names}_{i}",
        "times": 100,
        "tags": (wave_names, f"size_{i}"),
    }
    for _ in range(10)
    for i in range(2, 13, 2)
    for wave_names in ["trivial_paramagnet", "ghz", "neel"]
]
print(len(config_list))

180


The `.multiOutput` will return an id of this `multimanager` instance,
which can be used to get the results and post-process them.

Each `multimanager` will export the experiments in a folder you can specify
by setting `save_location` parameter with default location for current directory
where Python executed.
It will create a folder with the name of the `multimanager` instance,
and inside it will create a folder for storing each experiment data.

It will do firstly in the building process, but you can skip it by setting `skip_build_write=True` to save time.
After all experiments are executed, it will export secondly,
which can also be skipped by setting `skip_output_write=True` for no files output.


In [5]:
multi_exps1 = experiment_randomized.multiOutput(
    config_list,
    summoner_name="qurrent.randomized_measure",  # you can name it whatever you want
    multiprocess_build=True,
    # Using multiprocessing to build the experiments,
    # it will be faster but take all the CPU
    skip_build_write=True,
    # Skip the writing of the experiment as files during the build,
    save_location=".",
    # Save the experiment as files in the current directory
    multiprocess_write=True,
    # Writing the experiment as files using multiprocessing,
)
multi_exps1

| MultiManager building...
| qurrent.randomized_measure.001 is repeat location.
| qurrent.randomized_measure.002 is repeat location.
| qurrent.randomized_measure.003 is repeat location.
| qurrent.randomized_measure.004 is repeat location.
| qurrent.randomized_measure.005 is repeat location.
| qurrent.randomized_measure.006 is repeat location.
| Write "qurrent.randomized_measure.007", at location "qurrent.randomized_measure.007"


| 0/180   0%|          | - MultiManager building... - 00:00 < ?

| MultiOutput running...


| 0/180   0%|          | -  - 00:00 < ?

| Export multimanager...


...

| No quantity to export.
| Export multi.config.json for 35981502-1353-452d-b2f4-512c2f9f7f44


| 0/179 - Exporting experiments... - 00:00 < ?

| 0/180 - Loading file infomation... - 00:00 < ?

| Exporting file taglist...
| Exporting qurrent.randomized_measure.007/qurryinfo.json...
| Exporting qurrent.randomized_measure.007/qurryinfo.json done.


'35981502-1353-452d-b2f4-512c2f9f7f44'

You can check the result of `multiOutput` that we just executed by accessing the `.multimanagers`

In [6]:
experiment_randomized.multimanagers

MultiManagerContainer(num=1, {
  "35981502-1353-452d-b2f4-512c2f9f7f44":
    <MultiManager(name="qurrent.randomized_measure.007", jobstype="local", ..., exps_num=180)>,
})

In [7]:
experiment_randomized.multimanagers[multi_exps1]

<MultiManager(id="35981502-1353-452d-b2f4-512c2f9f7f44",
  name="qurrent.randomized_measure.007",
  tags=(),
  jobstype="local",
  pending_strategy="tags",
  last_events={
    'output.001': '2025-05-22 15:37:34',},
  exps_num=180)>

### d. Run post-processing at once


In [8]:
experiment_randomized.multiAnalysis(
    summoner_id=multi_exps1,
    skip_write=True,
    multiprocess_write=False,
    selected_qubits=[0, 1],
)

| 0/180 - Analysis:  - 00:00 < ?

| "report.001" has been completed.


'35981502-1353-452d-b2f4-512c2f9f7f44'

In [31]:
experiment_randomized.multimanagers[multi_exps1].quantity_container["report.001"]

TagList(list,
        {('trivial_paramagnet',
          'size_2'): [{'purity': np.float64(1.0944049453735352),
           'entropy': np.float64(-0.13014665457050778),
           'puritySD': np.float64(0.7180731985654023),
           'entropySD': np.float64(0.9465971868502426),
           'num_classical_registers': 2,
           'classical_registers': [0, 1],
           'classical_registers_actually': [0, 1],
           'all_system_source': 'independent',
           'purityAllSys': np.float64(1.0944049453735352),
           'entropyAllSys': np.float64(-0.13014665457050778),
           'puritySDAllSys': np.float64(0.7180731985654022),
           'entropySDAllSys': np.float64(0.9465971868502424),
           'num_classical_registers_all_sys': 2,
           'classical_registers_all_sys': None,
           'classical_registers_actually_all_sys': [0, 1],
           'errorRate': np.float64(-0.06107175087175273),
           'mitigatedPurity': np.float64(1.0),
           'mitigatedEntropy': np.fl

In [10]:
print("| Available results:")
for k, v in (
    experiment_randomized.multimanagers[multi_exps1]
    .quantity_container["report.001"]
    .items()
):
    print("| -", k, "with length", len(v))

| Available results:
| - ('trivial_paramagnet', 'size_2') with length 10
| - ('ghz', 'size_2') with length 10
| - ('neel', 'size_2') with length 10
| - ('trivial_paramagnet', 'size_4') with length 10
| - ('ghz', 'size_4') with length 10
| - ('neel', 'size_4') with length 10
| - ('trivial_paramagnet', 'size_6') with length 10
| - ('ghz', 'size_6') with length 10
| - ('neel', 'size_6') with length 10
| - ('trivial_paramagnet', 'size_8') with length 10
| - ('ghz', 'size_8') with length 10
| - ('neel', 'size_8') with length 10
| - ('trivial_paramagnet', 'size_10') with length 10
| - ('ghz', 'size_10') with length 10
| - ('neel', 'size_10') with length 10
| - ('trivial_paramagnet', 'size_12') with length 10
| - ('ghz', 'size_12') with length 10
| - ('neel', 'size_12') with length 10


In [14]:
experiment_randomized.multimanagers[multi_exps1].quantity_container["report.001"][
    ("trivial_paramagnet", "size_10")
][0]

{'purity': np.float64(1.0075217437744142),
 'entropy': np.float64(-0.010810974531916692),
 'puritySD': np.float64(0.6921363939986136),
 'entropySD': np.float64(0.9910870404640565),
 'num_classical_registers': 10,
 'classical_registers': [0, 1],
 'classical_registers_actually': [0, 1],
 'all_system_source': 'independent',
 'purityAllSys': np.float64(2.4932796287536623),
 'entropyAllSys': np.float64(-1.3180446944591888),
 'puritySDAllSys': np.float64(4.600867561501006),
 'entropySDAllSys': np.float64(2.662215957735322),
 'num_classical_registers_all_sys': 10,
 'classical_registers_all_sys': None,
 'classical_registers_actually_all_sys': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'errorRate': np.float64(-0.5794743857168532),
 'mitigatedPurity': np.float64(0.5536476529253226),
 'mitigatedEntropy': np.float64(0.8529599725768597),
 'counts_num': 100,
 'taking_time': 0.000435249,
 'taking_time_all_sys': 0.017614681,
 'counts_used': None,
 'input': {'num_qubits': 10,
  'selected_qubits': [0, 1],
  'regi

### e. Run post-processing at once with specific analysis arguments

At first, we need to get the each experiment's id in the `multimanager` instance.


In [15]:
expkeys_of_multi_exps1 = list(
    experiment_randomized.multimanagers[multi_exps1].exps.keys()
)
print(len(expkeys_of_multi_exps1))

180


1. If you want to run the post-processing for some specific experiments,
   for example, the first 3 experiments we get for the `multimanager` instance.


In [16]:
expkeys_of_multi_exps1

['8f0bf159-1787-46a6-b225-dc7a4111ebc4',
 '4c2a60fd-c40f-4134-bc41-e0f9244f41b6',
 'bbaafbdd-d755-464f-80c1-a38641aac8d2',
 '22adbd7e-1ad5-415e-9405-ff9b838a18a6',
 '3564a968-0311-44eb-a5d1-d13414b41ccf',
 'ed315ff2-348a-4652-8d5e-bf9ae9ac2982',
 'b1376166-a89e-4005-b18b-dec4e6c29618',
 'ae618ef6-e729-44ad-ba4f-e19fd649b7c4',
 '1af9dd54-b677-4f89-adc1-13efa0b3b4a2',
 'f9942bc8-7713-496c-8752-34a8bf648ec6',
 '2ae315de-1ca0-4901-be4b-700ddda196e1',
 'a6a4e9c5-72df-4864-9593-02d8b59fb2a0',
 'a32f444c-e043-42f0-a58b-830196246092',
 '05dfbca5-b008-459a-bc8e-75f62020dcbc',
 '8b855352-34eb-497c-9c32-f3fafc24edb5',
 '98c15a85-c807-4a65-a309-4a80e39845a1',
 'cf6aea1d-b421-4b7a-97d1-6e5fc256776d',
 '3f0c06b2-2f47-4bf1-b0ac-e7cdd0960fda',
 '11e6fec9-c3d7-4635-b054-7b983ae6a75c',
 'd7292912-95eb-4744-a8eb-18a3fb5e8c7e',
 'a8064c34-26ea-45d0-a258-eda6270fbe7a',
 'd499f7e9-b17a-4f70-af66-8da8708555cf',
 '6000eb3d-cfd3-49d6-ac40-b1513670540d',
 '7896b6be-f45f-4670-970c-85fea2cd6226',
 '60cf9b42-8973-

In [18]:
{
    k: (
        {
            "selected_qubits": [0, 1],
        }
        if idx < 3
        else False
    )
    for idx, k in enumerate(expkeys_of_multi_exps1)
}

{'8f0bf159-1787-46a6-b225-dc7a4111ebc4': {'selected_qubits': [0, 1]},
 '4c2a60fd-c40f-4134-bc41-e0f9244f41b6': {'selected_qubits': [0, 1]},
 'bbaafbdd-d755-464f-80c1-a38641aac8d2': {'selected_qubits': [0, 1]},
 '22adbd7e-1ad5-415e-9405-ff9b838a18a6': False,
 '3564a968-0311-44eb-a5d1-d13414b41ccf': False,
 'ed315ff2-348a-4652-8d5e-bf9ae9ac2982': False,
 'b1376166-a89e-4005-b18b-dec4e6c29618': False,
 'ae618ef6-e729-44ad-ba4f-e19fd649b7c4': False,
 '1af9dd54-b677-4f89-adc1-13efa0b3b4a2': False,
 'f9942bc8-7713-496c-8752-34a8bf648ec6': False,
 '2ae315de-1ca0-4901-be4b-700ddda196e1': False,
 'a6a4e9c5-72df-4864-9593-02d8b59fb2a0': False,
 'a32f444c-e043-42f0-a58b-830196246092': False,
 '05dfbca5-b008-459a-bc8e-75f62020dcbc': False,
 '8b855352-34eb-497c-9c32-f3fafc24edb5': False,
 '98c15a85-c807-4a65-a309-4a80e39845a1': False,
 'cf6aea1d-b421-4b7a-97d1-6e5fc256776d': False,
 '3f0c06b2-2f47-4bf1-b0ac-e7cdd0960fda': False,
 '11e6fec9-c3d7-4635-b054-7b983ae6a75c': False,
 'd7292912-95eb-4744-a

In [19]:
experiment_randomized.multiAnalysis(
    summoner_id=multi_exps1,
    analysis_name="first_3",
    skip_write=True,
    multiprocess_write=False,
    specific_analysis_args={
        k: (
            {
                "selected_qubits": [0, 1],
            }
            if idx < 3
            else False
        )
        for idx, k in enumerate(expkeys_of_multi_exps1)
    },
)

| 0/180 - Analysis:  - 00:00 < ?

| "first_3.002" has been completed.


'35981502-1353-452d-b2f4-512c2f9f7f44'

In [20]:
print("| Available results:")
print(
    "| length:",
    sum(
        len(v)
        for v in experiment_randomized.multimanagers[multi_exps1]
        .quantity_container["first_3.002"]
        .values()
    ),
)

| Available results:
| length: 3


2. Or manually specify all the analysis arguments for each experiment.


In [21]:
experiment_randomized.multiAnalysis(
    summoner_id=multi_exps1,
    skip_write=False,
    analysis_name="all_manual",
    multiprocess_write=True,
    specific_analysis_args={
        k: {
            "selected_qubits": [0, 1],  # selected qubits for the analysis
        }
        for idx, k in enumerate(expkeys_of_multi_exps1)
    },
)

| 0/180 - Analysis:  - 00:00 < ?

| "all_manual.003" has been completed.
| Export multimanager...


| 0/9 - Exporting MultiManager content... - 00:00 < ?

| 0/3 - exporting quantity - 00:00 < ?

| Export multi.config.json for 35981502-1353-452d-b2f4-512c2f9f7f44


| 0/179 - Exporting experiments... - 00:00 < ?

| 0/180 - Loading file infomation... - 00:00 < ?

| Exporting file taglist...
| Exporting qurrent.randomized_measure.007/qurryinfo.json...
| Exporting qurrent.randomized_measure.007/qurryinfo.json done.


'35981502-1353-452d-b2f4-512c2f9f7f44'

In [28]:
experiment_randomized.multimanagers[multi_exps1].quantity_container.keys()

dict_keys(['report.001', 'first_3.002', 'all_manual.003'])

In [22]:
print("| Available results:")
print(
    "| length:",
    sum(
        len(v)
        for v in experiment_randomized.multimanagers[multi_exps1]
        .quantity_container["all_manual.003"]
        .values()
    ),
)

| Available results:
| length: 180


### f. Read exported multimanager data


In [23]:
multi_exps1_reades = experiment_randomized.multiRead(
    save_location=".",
    summoner_name="qurrent.randomized_measure.001",
)

| Retrieve qurrent.randomized_measure.001...
| at: qurrent.randomized_measure.001


| 0/180   0%|          | - Loading 180 experiments ... - 00:00 < ?

In [25]:
experiment_randomized.multimanagers["d8146e57-30c9-44c9-a80b-dd761ac21252"].exps

ExperimentContainer(num=180, {
  '402f1713-6463-4f22-b804-0fd7318c1cff': <EntropyMeasureRandomizedExperiment(EntropyMeasureRandomizedArguments(exp_name='qurrent.randomized_measure.001.N_U_100.qurrent_randomized', times=100, qubits_measured=[0, 1, 2, 3], registers_mapping={0: 0, 1: 1, 2: 2, 3: 3}, actual_num_qubits=4, unitary_located=[0, 1, 2, 3], random_unitary_seeds=None), Commonparams(exp_id='402f1713-6463-4f22-b804-0fd7318c1cff', target_keys=['ghz_4'], shots=1024, backend='basic_simulator', run_args={}, transpile_args={}, tags=('ghz', 'size_4'), default_analysis=[], save_location='.', filename='index=22.id=402f1713-6463-4f22-b804-0fd7318c1cff', files={}, serial=22, summoner_id='d8146e57-30c9-44c9-a80b-dd761ac21252', summoner_name='qurrent.randomized_measure.001', datetimes=DatetimeDict({'build': '2025-05-22 14:38:16', 'run.001': '2025-05-22 14:38:44'})), unused_args_num=2, analysis_num=2)>,
  '57f39bbd-ee30-43ca-84e3-1fe7b97a5c6a': <EntropyMeasureRandomizedExperiment(EntropyMeasureR

In [16]:
expkeys_of_multi_exps1

['d7e5bde9-35d6-4480-846c-1ce135f12a22',
 '4108b5fd-de55-4671-a688-f36a8a8d7e7c',
 '11108337-10bd-437c-b61b-338e44f81c8e',
 'c4eabf32-d21f-4481-873f-7d81a6e7eb6c',
 'e78b4f39-d272-43d2-bfca-2b0590f047a1',
 '81c4a84d-c7a5-4091-b199-bf2415c03be2',
 '060c5100-2de1-46c4-9b32-8fcf3e3df3cc',
 '713153be-a43b-4e78-8183-7dafafb3259d',
 '57f39bbd-ee30-43ca-84e3-1fe7b97a5c6a',
 'b9e62941-da32-4cae-8dde-cb1fe132f7c0',
 '402f1713-6463-4f22-b804-0fd7318c1cff',
 '496d4543-8025-4e94-b6e2-a70254ab7588',
 'ed6f02de-ed34-4658-a0ca-969e29b8dd71',
 '957fee73-38ce-4947-beea-8b260224b296',
 '1cbdbe12-6cb5-4db5-badc-440939f5ebf8',
 '8e306ea1-1bf4-4bdf-a43a-64d9c3fb28ae',
 '7d059cc9-11e2-41b7-9555-8c31b695bab0',
 'd2e3e51a-37ec-4ee8-a308-fe91a80b33d5',
 'd973016c-a7f3-4227-a86a-241d99e2a766',
 'bee05788-253a-491f-be81-a3312c2a6db7',
 '8db85638-c3e0-4747-bb26-9f8b24a6f45e',
 'f9aef5b8-6712-4914-927b-6f8c1f187290',
 'f61b9335-553b-4d63-9ae3-d9be46e41a88',
 '6a72e672-ccfa-46d5-8a1b-8ef5264d7208',
 'ed3ffd03-d6da-

---

### Post-Process Availablities and Version Info

In [17]:
from qurry.process import AVAIBILITY_STATESHEET

AVAIBILITY_STATESHEET

 | Qurry version: 0.12.2.dev2
---------------------------------------------------------------------------
 ### Qurry Post-Processing
   - Backend Availability ................... Python Cython Rust   JAX   
 - randomized_measure
   - entangled_entropy.entropy_core_2 ....... Yes    Depr.  Yes    No    
   - entangle_entropy.purity_cell_2 ......... Yes    Depr.  Yes    No    
   - entangled_entropy_v1.entropy_core ...... Yes    Depr.  Yes    No    
   - entangle_entropy_v1.purity_cell ........ Yes    Depr.  Yes    No    
   - wavefunction_overlap.echo_core_2 ....... Yes    Depr.  Yes    No    
   - wavefunction_overlap.echo_cell_2 ....... Yes    Depr.  Yes    No    
   - wavefunction_overlap_v1.echo_core ...... Yes    Depr.  Yes    No    
   - wavefunction_overlap_v1.echo_cell ...... Yes    Depr.  Yes    No    
 - hadamard_test
   - purity_echo_core ....................... Yes    No     Yes    No    
 - magnet_square
   - magnsq_core ............................ Yes    No     No     No  